In [1]:
Pkg.resolve() # Installiert alle dependiencies aus Project.toml auf dem Rechner. Muss nur einmal ausgeführt werden.

LoadError: UndefVarError: Pkg not defined

In [2]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.status()

  Activating project at `C:\Users\Chris\Documents\AMO-single`


Status `C:\Users\Chris\Documents\AMO-single\Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [485130c0] BilevelJuMP v0.5.2
  [a076750e] CPLEX v0.9.7
  [336ed68f] CSV v0.10.9
  [aaaa29a8] Clustering v0.14.3
⌃ [a9b2a840] Complementarity v0.8.4
  [a93c6f00] DataFrames v1.5.0
  [864edb3b] DataStructures v0.18.13
  [b4f34e82] Distances v0.10.7
  [31c24e10] Distributions v0.25.86
  [5789e2e9] FileIO v1.16.0
  [87dc4568] HiGHS v1.4.3
  [7073ff75] IJulia v1.24.0
  [916415d5] Images v0.25.2
  [b6b21f68] Ipopt v1.2.0
  [c8e1da08] IterTools v1.4.0
⌅ [4076af6c] JuMP v1.1.1
  [b8f27783] MathOptInterface v1.13.1
  [8314cec4] PGFPlotsX v1.5.3
  [f0f68f2c] PlotlyJS v0.18.10
  [91a5bcdd] Plots v1.38.6
  [67491407] Polyhedra v0.7.6
  [ce6b1742] RDatasets v0.7.7
  [f3b207a7] StatsPlots v0.15.4
  [fdbf4ff8] XLSX v0.8.4
  [9a3f8284] Random
  [10745b16] Statistics
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To 

In [10]:
# On Windows, this might be
ENV["CPLEX_STUDIO_BINARIES"] = "C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community2211\\cplex\\bin\\x64_win64"
# On OSX, this might be

import Pkg
Pkg.add("CPLEX")
Pkg.build("CPLEX")

   Resolving package versions...
  No Changes to `C:\Users\Chris\Documents\AMO-single\Project.toml`
  No Changes to `C:\Users\Chris\Documents\AMO-single\Manifest.toml`
    Building CPLEX → `C:\Users\Chris\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\2b1c524b1c9dc3ef283761737273a93470a5caeb\build.log`


In [15]:
Pkg.add("BilevelJuMP")

   Resolving package versions...
   Installed GR_jll ─────────────── v0.71.7+0
   Installed JpegTurbo_jll ──────── v2.1.91+0
   Installed FFTW ───────────────── v1.6.0
   Installed ImageSegmentation ──── v1.8.0
   Installed StatsFuns ──────────── v1.3.0
   Installed JpegTurbo ──────────── v0.1.2
   Installed MutableArithmetics ─── v1.2.3
   Installed Dualization ────────── v0.5.6
   Installed PDMats ─────────────── v0.11.17
   Installed IrrationalConstants ── v0.2.2
   Installed MUMPS_seq_jll ──────── v500.500.101+0
   Installed FiniteDiff ─────────── v2.18.0
   Installed StaticArrays ───────── v1.5.16
   Installed JLD2 ───────────────── v0.4.31
   Installed DataFrames ─────────── v1.5.0
   Installed SentinelArrays ─────── v1.3.18
   Installed SpecialFunctions ───── v2.2.0
   Installed NaNMath ────────────── v1.0.2
   Installed Ipopt_jll ──────────── v300.1400.403+1
   Installed GR ─────────────────── v0.71.7
   Installed JuMP ───────────────── v1.1.1
   Installed DiffRules ───────────

In [3]:
using CPLEX
using BilevelJuMP
using JuMP
using BenchmarkTools

[ Info: Precompiling BilevelJuMP [485130c0-026e-11ea-0f1a-6992cd14145c]


KKT Conditions used in this case   
For convex problems - at the same time global optimum
  
KKT can be local optima  
KKT first order conditions

In [4]:
bm = BilevelModel(CPLEX.Optimizer)

An Abstract JuMP Model
Feasibility problem with:
Variables: 0
Upper Constraints: 0
Lower Constraints: 0
Bilevel Model
Solution method: BilevelJuMP.SOS1Mode{Float64}()
Solver name: CPLEX

In [25]:
unregister(bm, :x)
unregister(bm, :y)

#Variables
@variable(Upper(bm), 0 <= x <= 4)
@variable(Lower(bm), y >= 0)

#Upper level 
@constraint(Upper(bm), 2x+y <= 8)
@objective(Upper(bm), Min, x+4y)

#Lower level
@constraint(Lower(bm), x+y <= 7)
@objective(Lower(bm), Min, -x-y)

optimize!(bm)

Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
MIP Presolve eliminated 2 redundant SOS constraints.
MIP Presolve eliminated 1 redundant SOS constraints.
Tried aggregator 2 times.
MIP Presolve eliminated 6 rows and 11 columns.
Aggregator did 1 substitutions.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.03 sec. (0.02 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.03 sec. (0.02 ticks)


In [26]:
@show value(x)
@show value(y)

value(x) = 1.0
value(y) = 6.0


6.0

Using other mode in terms of solution speed

In [30]:
function bilevel_fn(; mode = BilevelJuMP.SOS1Mode())
    bm = BilevelModel(CPLEX.Optimizer)
    set_silent(bm) # uselful for benchmarking

    unregister(bm, :x)
    unregister(bm, :y)

    #Variables
    @variable(Upper(bm), 0 <= x <= 4)
    @variable(Lower(bm), y >= 0)

    #Upper level 
    @constraint(Upper(bm), 2x+y <= 8)
    @objective(Upper(bm), Min, x+4y)

    #Lower level
    @constraint(Lower(bm), x+y <= 7)
    @objective(Lower(bm), Min, -x-y)

    optimize!(bm)
    
    return value(x), value(y)
end

bilevel_fn (generic function with 1 method)

In [31]:
bilevel_fn()

(1.0, 6.0)

In [32]:
@benchmark bilevel_fn()

BenchmarkTools.Trial: 584 samples with 1 evaluation.
 Range (min … max):  7.124 ms … 28.693 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.255 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.528 ms ±  1.177 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▂▃▇██▂▄▂▂▁ ▁                                         
  ▂▂▁▃▅▄███████████▇█▇▇▆▅▅▆▅▄▅▅▄▄▄▄▃▄▄▃▃▂▂▁▂▃▂▁▁▁▂▁▂▂▂▁▁▁▂▂▂ ▄
  7.12 ms        Histogram: frequency by time        11.8 ms <

 Memory estimate: 145.02 KiB, allocs estimate: 2903.

In [33]:
@benchmark bilevel_fn(mode=BilevelJuMP.IndicatorMode())

BenchmarkTools.Trial: 624 samples with 1 evaluation.
 Range (min … max):  7.127 ms …  10.127 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.913 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.985 ms ± 423.584 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▃▂▆▇▇█ ▅▇█▁    ▁                                   
  ▃▁▁▃▃▃▄▆▄▆██████████████▇█▅▆▅▄▃▄▄▃▃▂▃▃▂▃▂▂▁▁▂▃▂▃▂▃▁▁▂▂▂▂▁▂▂ ▄
  7.13 ms         Histogram: frequency by time        9.76 ms <

 Memory estimate: 145.02 KiB, allocs estimate: 2903.

## Big M Method

In [35]:
@benchmark bilevel_fn(mode=BilevelJuMP.FortunyAmatMcCarlMode(
        primal_big_M = 1_000_000, 
        dual_big_M = 1_000_000))

BenchmarkTools.Trial: 620 samples with 1 evaluation.
 Range (min … max):  6.811 ms …  10.751 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.928 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.026 ms ± 587.111 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

               ▁▂█▃▅▄█▅▆▂ ▂▂                                   
  ▃▁▃▃▄▄▃▄▄▇▆▅▆█████████████▇▄▆▃▅▄▄▃▃▃▄▄▂▃▃▃▃▃▂▃▃▃▃▃▃▂▃▃▁▂▁▁▂ ▄
  6.81 ms         Histogram: frequency by time        10.2 ms <

 Memory estimate: 147.12 KiB, allocs estimate: 2919.

# Generation Expansion Planning

In [36]:
G = 1:1 # set of generators
C = 1:1 # set of candiates
D = 1:1 # demand
O = 1:2 # outcomes
Q = 1:6 # steps in potential new installed capacities (100MW steps of newly built capacities)

1:6

In [62]:
ic = [70_000.0] # investment cost per MW - 1 x C

pd = [290.0 550.0] # D x O

edata = [400.0 35.0] # G x (capacity, cost)

CandiData = [0.0 100.0 200.0 300.0 400.0 500.0] # C x Q

cdata = [25.0] # C x (Cost)

ρ = [6000 2760] # O \Rho

M = 300_000 


300000

In [71]:
gep = Model(CPLEX.Optimizer)

#set_silent(gep) # use when benchmarking!

@variables(gep, begin
    pge[g in G, o in O] >= 0
    pgc[c in C, o in O] >= 0
    pcmax[c in C] >= 0
    λ[o in O] >= 0
    uOp[c in C, q in Q], Bin
    μ_emax[g in G, o in O] >= 0
    μ_cmax[c in C, o in O] >= 0
    zaux[c in C, q in Q, o in O]
    zaux2[c in C, q in Q, o in O]
end)

@constraint(gep, EQ7B[c in C], 
    pcmax[c] == sum(uOp[c,q]*CandiData[c, q] for q in Q))

@constraint(gep, EQ7C[c in C],
    sum(uOp[c, :]) == 1)

@constraint(gep, EQ7E[o in O],
    sum(pge[g, o] for g in G) + 
    sum(pgc[c, o] for c in C) ==
    sum(pd[d, o] for d in D))

@constraint(gep, EQ7F[g in G, o in O],
    pge[g,o] <= edata[g, 1])

@constraint(gep, EQ7G[c in C, o in O],
    pgc[c, o] <= pcmax[c])

@constraint(gep, EQ7H[g in G, o in O],
    edata[g, 2] - λ[o] + μ_emax[g, o] >= 0)

@constraint(gep, EQ7I[c in C, o in O],
    cdata[c, 1] - λ[o] + μ_cmax[c, o] >= 0)

@constraint(gep, EQ7L[o in O],
    sum(edata[g, 2] * pge[g,o] for g in G) +
    sum(cdata[c, 1] * pgc[c,o] for c in C) ==
    λ[o] * sum(pd[d,o] for d in D) - 
    sum(μ_emax[g,o] * edata[g, 1] for g in G) -
    sum(sum(zaux[c,q,o] for q in Q) for c in C))

@constraint(gep, EQ7M[c in C, q in Q, o in O],
    zaux[c,q,o] == μ_cmax[c,o]*CandiData[c,q]-zaux2[c,q,o])

@constraint(gep, EQ7Na[c in C,q in Q,o in O],
    0 <= zaux[c,q,o])

@constraint(gep, EQ7Nb[c in C,q in Q,o in O],
    zaux[c,q,o] <= uOp[c,q] * M)

@constraint(gep, EQ7Oa[c in C,q in Q,o in O],
    0 <= zaux2[c,q,o])

@constraint(gep, EQ7Ob[c in C,q in Q,o in O],
    zaux2[c,q,o] <= (1-uOp[c,q]) * M)

@objective(gep, Min,
    sum(
        ρ[o] * (sum(edata[g, 2]*pge[g,o] for g in G) + 
        sum(cdata[c, 1]*pgc[c,o] for c in C))
    for o in O) + 
    sum(ic[c]*pcmax[c] for c in C))

210000 pge[1,1] + 150000 pgc[1,1] + 96600 pge[1,2] + 69000 pgc[1,2] + 70000 pcmax[1]

In [64]:
print(gep)

In [72]:
optimize!(gep)

Version identifier: 22.1.1.0 | 2022-11-26 | 9160aff4d
Tried aggregator 2 times.
MIP Presolve eliminated 38 rows and 10 columns.
MIP Presolve modified 67 coefficients.
Aggregator did 7 substitutions.
Reduced MIP has 29 rows, 24 columns, and 95 nonzeros.
Reduced MIP has 5 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.17 ticks)
Found incumbent of value 1.1051000e+08 after 0.02 sec. (0.22 ticks)
Probing fixed 1 vars, tightened 17 bounds.
Probing time = 0.02 sec. (0.45 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 1 rows and 2 columns.
MIP Presolve modified 21 coefficients.
Reduced MIP has 28 rows, 22 columns, and 87 nonzeros.
Reduced MIP has 4 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing fixed 0 vars, tightened 4 bounds.
Probing time = 0.00 sec. (0.12 ticks)
Clique table members: 1.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel m

In [73]:
termination_status(gep)

OPTIMAL::TerminationStatusCode = 1

In [74]:
value.(pge)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:1
    Dimension 2, 1:2
And data, a 1×2 Matrix{Float64}:
 0.0  250.0

In [75]:
value.(pgc)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:1
    Dimension 2, 1:2
And data, a 1×2 Matrix{Float64}:
 290.0  300.0

## manual with indicator mode

In [76]:
gep2 = Model(CPLEX.Optimizer)
set_silent(gep2) # use when benchmarking!

@variables(gep2, begin
    0 <= pge[g in G, o in O] <= edata[g, 1]
    pgc[c in C, o in O] >= 0
    pcmax[c in C] >= 0
    λ[o in O] >= 0
    uOp[c in C, q in Q], Bin
    μ_emax[g in G, o in O] >= 0
    μ_cmax[c in C, o in O] >= 0
    zaux[c in C, q in Q, o in O] >= 0
    zaux2[c in C, q in Q, o in O] >= 0
end)

@constraint(gep2, EQ7B[c in C], 
    pcmax[c] == sum(uOp[c,q]*CandiData[c, q] for q in Q))

@constraint(gep2, EQ7C[c in C],
    sum(uOp[c, :]) == 1)

@constraint(gep2, EQ7E[o in O],
    sum(pge[g, o] for g in G) + 
    sum(pgc[c, o] for c in C) ==
    sum(pd[d, o] for d in D))

@constraint(gep2, EQ7G[c in C, o in O],
    pgc[c, o] <= pcmax[c])

@constraint(gep2, EQ7H[g in G, o in O],
    edata[g, 2] - λ[o] + μ_emax[g, o] >= 0)

@constraint(gep2, EQ7I[c in C, o in O],
    cdata[c, 1] - λ[o] + μ_cmax[c, o] >= 0)

@constraint(gep2, EQ7L[o in O],
    sum(edata[g, 2] * pge[g,o] for g in G) +
    sum(cdata[c, 1] * pgc[c,o] for c in C) ==
    λ[o] * sum(pd[d,o] for d in D) - 
    sum(μ_emax[g,o] * edata[g, 1] for g in G) -
    sum(sum(zaux[c,q,o] for q in Q) for c in C))

@constraint(gep2, EQ7M[c in C, q in Q, o in O],
    zaux[c,q,o] == μ_cmax[c,o]*CandiData[c,q]-zaux2[c,q,o])

@constraint(gep2, EQ7Nb[c in C,q in Q,o in O],
    !uOp[c,q] => {zaux[c,q,o] == 0})

@constraint(gep2, EQ7Ob[c in C,q in Q,o in O],
    uOp[c,q] => {zaux2[c,q,o] == 0})

@objective(gep2, Min,
    sum(
        ρ[o] * (sum(edata[g, 2]*pge[g,o] for g in G) + 
        sum(cdata[c, 1]*pgc[c,o] for c in C))
    for o in O) + 
    sum(ic[c]*pcmax[c] for c in C))

210000 pge[1,1] + 150000 pgc[1,1] + 96600 pge[1,2] + 69000 pgc[1,2] + 70000 pcmax[1]

In [77]:
optimize!(gep2)

In [78]:
value.(uOp)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:1
    Dimension 2, 1:6
And data, a 1×6 Matrix{Float64}:
 -0.0  0.0  0.0  1.0  0.0  0.0

## Randomness

In [79]:
using Distributions, Plots, PGFPlotsX
pgfplotsx()

[ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]


Plots.PGFPlotsXBackend()

In [80]:
wind = Weibull(2, 5.0)

Weibull{Float64}(α=2.0, θ=5.0)

In [81]:
x = 0.0:0.01:12.0

plot(x, pdf(wind, x))

LoadError: MethodError: pdf(::Weibull{Float64}, ::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}) is ambiguous. Candidates:
  pdf(d::Distribution{ArrayLikeVariate{N}}, x::AbstractArray{<:Real, M}) where {N, M} in Distributions at C:\Users\Chris\.julia\packages\Distributions\YQQXX\src\common.jl:275
  pdf(d::UnivariateDistribution, X::AbstractArray) in Distributions at deprecated.jl:70
Possible fix, define
  pdf(::UnivariateDistribution, ::AbstractArray{T, M} where T<:Real) where M

In [82]:
N = 100_000

sample = rand(wind, N)

100000-element Vector{Float64}:
  1.0970403944929974
  4.892680934075254
  3.219690794022636
 10.665595001779455
  2.4480062192113934
  5.767006351608433
  3.410973272285978
  6.334284108121487
  5.7439639267398075
  3.342360107758573
  5.411815221523595
  8.476823608356693
  6.046697454246157
  ⋮
  5.182772854148729
  2.662596028809289
  7.932446762558833
  5.397656388016669
  3.3098320196424336
  2.6959176495402137
  8.359515331725612
  2.280884162578896
  6.374202903278201
 10.031242541417658
  3.4440865848306363
  4.154944232656937

In [83]:
histogram(
    sample, 
    label="Sample N = "*string(N),
    xlabel="Wind speed",
    ylabel="Count",
    lw=0.2,
    c=:lightgray,
    xlims = (0, max(sample...)),
    ylims = (0, 3500),
    legend=:topright,
    size=(500,500),
    xtickfontsize=12,
    ytickfontsize=12,
    xlabelfontsize=14,
    ylabelfontsize=14,
    legendfontsize=12,
)

PGFPlotsX.MissingExternalProgramError: No LaTeX installation found, figures will not be generated. Make sure either pdflatex, xelatex or lualatex are installed and that the PATH variable is correctly set.